# 机场廊桥派工人员模拟环境

机场廊桥过程中涉及以下智能体：
1. 航班 Flights，航班本身具有发车时间和到达时间的属性，由此诞生出任务的概念
2. 勤务人员 Individual，机场具有的人员的资质属性、位置等，动态更新是否分配任务

在智能体下会产生不同的传输子类
1. 任务类 Task，介绍航班出港或者入港带来的状态更新
2. 航司类 AviationCompany, 介绍航司的基本信息

同时存在环境 Model
1. 环境作用：规范仿真过程中的时间和空间模块
2. 对观察的信息作为一定的输入，不同更新 model 的状态

In [3]:
import mesa
import pandas as pd
import numpy as np

# 航司类

In [74]:
class Task():
    def __init__(self) -> None:
        # 任务需要有位置、任务选项
        self.location = None
        self.time = None
        self.taskList = []
        
class AviationCompany():
    def __init__(self):
        self.name = None
        self.codingId = None
        self.terminal = 'T2' # set for T2 or S2
        self.isYiBan = False
        self.isFangXing = False
        self.isWeiXiu = False
        self.isZhongwen = False
        self.isYingwen = False
    
    def create_task(self,time = None):
        newTask = Task()
        newTask.location = self.terminal
        newTask.time = time
        newTask.taskList = [self.isYiBan,self.isFangXing,self.isWeiXiu,self.isZhongwen,self.isYingwen]
        return newTask

class ACsets():
    def __init__(self):
        self.dfCompany = pd.DataFrame(columns=['name', 'codingId', 'terminal', 'isYiBan', 'isFangXing', 'isWeiXiu', 'isZhongWen', 'isYingWen'])
        self.companySets = set()
    
    def login(self,path):
        # 根据 path 提供的航司资质来注册航司列表
        df_com = pd.read_excel(path)
        df_com.drop(['航班类型'], axis=1, inplace=True)
        # 重命名列
        df_com.rename(columns={'航空公司':'name', '二字代码':'codingId', '停机坪':'terminal','一般勤务':'isYiBan','放行机务':'isFangXing','维修机务':'isWeiXiu','英文耳机':'isYingWen','中文耳机':'isZhongWen'}, inplace=True)
        # 重命列
        self.dfCompany = df_com
        self.companySets = set(df_com['codingId'])
    
    def get_company(self,codingID):
        # 根据二字代码获取航司信息
        avi = AviationCompany()
        temp_row = self.dfCompany[self.dfCompany['codingId'] == codingID]
        if temp_row.empty:
            raise 'The company is not in the list'
        else:
            avi.name = temp_row['name'].values[0]
            avi.codingId = temp_row['codingId'].values[0]
            avi.terminal = temp_row['terminal'].values[0]
            avi.isYiBan = temp_row['isYiBan'].values[0]
            avi.isFangXing = temp_row['isFangXing'].values[0]
            avi.isWeiXiu = temp_row['isWeiXiu'].values[0]
            avi.isZhongwen = temp_row['isZhongWen'].values[0]
            avi.isYingwen = temp_row['isYingWen'].values[0]
            return avi

In [75]:
path =  '../dataset/new_aviationCompany.xlsx'
acsets = ACsets()
acsets.login(path)

In [76]:
print(acsets.get_company('CA').create_task().taskList)

[0, 0, 0, 0, 0]


# 机场环境类 Model


In [77]:
# 每 20min 获取任务信息，以实际到达时间为准
flights = pd.read_excel('../dataset/flights_obs.xlsx')
flights['actual_time'] = pd.to_timedelta(flights['actual_time'])
flights['actual_datetime'] = flights['actual_date'] + flights['actual_time']
flights.sort_values(by='actual_datetime', inplace=True)
flights.reset_index(drop=True, inplace=True)
flights.drop(['actual_time', 'actual_date'], axis=1, inplace=True)
print(flights.head(5))  

  航班类别       机号    机型   机位  plan_date plan_time 航空公司     type  \
0  PAX    9MXXU  A333   71 2024-02-01  00:20:00   D7  inbound   
1  PAX  RPC4124  A21N   87 2024-02-01  00:05:00   5J  inbound   
2  PAX    B323D  A20N   95 2024-02-01  00:30:00   HO  inbound   
3  CGO   JA601F  B763  617 2024-02-01  00:40:00   NH  inbound   
4  PAX   JA212P  A20N   79 2024-02-01  00:20:00   MM  inbound   

      actual_datetime  
0 2024-01-31 23:43:00  
1 2024-01-31 23:49:00  
2 2024-02-01 00:07:00  
3 2024-02-01 00:14:00  
4 2024-02-01 00:18:00  


In [78]:
# 计算 2024-01-01 00:00:00 15 分钟之内的任务
start_time = pd.to_datetime('2024-02-01 00:00:00')
end_time = start_time + pd.Timedelta('15 min')
print(start_time, end_time)
print(flights[(flights['actual_datetime'] >= start_time) & (flights['actual_datetime'] <= end_time)])

2024-02-01 00:00:00 2024-02-01 00:15:00
  航班类别      机号    机型   机位  plan_date plan_time 航空公司     type  \
2  PAX   B323D  A20N   95 2024-02-01  00:30:00   HO  inbound   
3  CGO  JA601F  B763  617 2024-02-01  00:40:00   NH  inbound   

      actual_datetime  
2 2024-02-01 00:07:00  
3 2024-02-01 00:14:00  


In [111]:
class Flights:
    def __init__(self) -> None:
        self.location = 0
        self.planDate = 0
        self.planTime = 0
        self.actualDate = 0
        self.actualTime = 0
        self.ac = None
        self.boundType = None
    
    def update(self,**kwargs):
        for key,value in kwargs.items():
            if hasattr(self,key):
                # hasattr(self,key) 判断是否有这个属性
                # setattr(self,key,value) 设置属性
                setattr(self,key,value)
            else:
                print('No such attribute')
    
    def from_row(self,row):
        self.location = row['机位']
        self.planDate = row['plan_date']
        self.planTime = row['plan_time']
        self.actualDateTime = row['actual_datetime']
        self.ac = acsets.get_company(row['航空公司'])
        self.boundType = row['type']
    
    def create_task(self):
        return self.ac.create_task(self.actualDateTime)

class TaskSet:
    def __init__(self) -> None:
        self.tasks = []
    
    def add_task(self,task):
        self.tasks.append(task)
    
    def get_tasks(self):
        return self.tasks
    
class FlightsSet:
    def __init__(self) -> None:
        self.df_flights = None
        self.Margin = 15
        self.list_flights = []
        self.taskSet = TaskSet()
        self.index = 0
    
    def login(self,path):
        # path = '../dataset/flights_obs.xlsx'
        flights = pd.read_excel(path)
        flights['actual_time'] = pd.to_timedelta(flights['actual_time'])
        flights['actual_datetime'] = flights['actual_date'] + flights['actual_time']
        flights.sort_values(by='actual_datetime', inplace=True)
        flights.reset_index(drop=True, inplace=True)
        flights.drop(['actual_time', 'actual_date'], axis=1, inplace=True)
        flights.dropna(inplace=True)
        self.df_flights = flights
    
    def filterByAC(self,codingId):
        self.df_flights = self.df_flights[self.df_flights['航空公司'].isin(codingId)]
        self.df_flights.reset_index(drop=True, inplace=True)
    
    def get_near_flights(self,datetime):
        ''' 
        获取距离当前 date、now ，在 margin 间隔内的航班信息，构造即将降落航班的属性
        创建任务到任务集合内
        '''
        while self.df_flights['actual_datetime'][self.index] <= datetime+pd.Timedelta(f'{self.Margin} min'):
            flight = Flights()
            flight.from_row(self.df_flights.iloc[self.index])
            self.list_flights.append(flight)
            self.index += 1
        return self.list_flights

    def add_flight(self,flight):
        self.flights.append(flight)
    
    def get_flights(self,idx):
        return Flights().from_row(self.df_flights.iloc[idx])

In [112]:
fs = FlightsSet()
fs.login('../dataset/flights_obs.xlsx')
fs.filterByAC(acsets.dfCompany['codingId'])

In [113]:
begin = fs.df_flights['actual_datetime'][0]
print(begin)

2024-01-31 23:43:00


In [114]:
def match_alor(list_flights):
    if len(list_flights)>4:
        complete_task = list_flights.pop(0).create_task()
        print(complete_task.time)
        print(complete_task.location)
        print(complete_task.taskList)
        print('Done')
    return list_flights

In [117]:
while fs.index < 20:
    lfs = fs.get_near_flights(begin)
    lfs = match_alor(lfs)
    begin += pd.Timedelta('2min')

2024-02-01 02:37:00
T2
[1, 0, 0, 0, 0]
Done
2024-02-01 03:44:00
S2
[0, 0, 0, 0, 0]
Done
2024-02-01 04:02:00
T2
[1, 0, 0, 0, 1]
Done
2024-02-01 04:21:00
T2
[0, 0, 0, 0, 0]
Done
2024-02-01 04:35:00
S2
[0, 0, 0, 0, 0]
Done
2024-02-01 04:39:00
T2
[0, 0, 0, 0, 0]
Done
2024-02-01 04:45:00
T2
[1, 0, 0, 0, 0]
Done
2024-02-01 04:53:00
S2
[0, 0, 0, 0, 0]
Done
2024-02-01 05:02:00
T2
[1, 0, 0, 0, 1]
Done
2024-02-01 05:20:00
T2
[1, 0, 0, 0, 0]
Done


In [27]:
class TaskSets():
    def __init__(self):
        # 新增 task
        # 删除 task 
        # 获取 task 的统计信息
        self.dfTask = pd.DataFrame(columns=['location', 'time', 'taskList'])
        self.taskSets = set()
    
    def login(self,path):
        pass 

    def append_task(self,task):
        self.dfTask = self.dfTask.append({'location': task.location, 'time': task.time, 'taskList': task.taskList}, ignore_index=True)
        self.taskSets.add(task)
    
    def len(self):
        return len(self.taskSets)

In [25]:
class Airports(mesa.Model):
    def __init__(self, num_airports, num_flights):
        # TODO 暂时未完成
        super().__init__()
        self.num_airports = num_airports
        self.num_flights = num_flights
        self.airports = pd.DataFrame(columns=['airport_id', 'x', 'y'])
        self.flights = pd.DataFrame(columns=['flight_id', 'origin', 'destination', 'departure_time', 'arrival_time'])
        self.airport_id = 0
        self.flight_id = 0
        self.create_airports()
        self.create_flights()
    
    def step(self) -> None:
        return super().step()
    
    def create_airports(self) -> None:
        for i in range(self.num_airports):
            # Create a new airport with a random x and y coordinate
            new_row = {'airport_id': self.airport_id, 'x': np.random.uniform(0, 100), 'y': np.random.uniform(0, 100)}
            self.airports = pd.concat([self.airports, pd.DataFrame(new_row, index=[0])], ignore_index=True)
            self.airport_id += 1
            
    def create_flights(self) -> None:
        for i in range(self.num_flights):
            origin = np.random.choice(self.airports['airport_id'])
            destination = np.random.choice(self.airports['airport_id'])
            while destination == origin:
                destination = np.random.choice(self.airports['airport_id'])
            departure_time = np.random.uniform(0, 24)
            arrival_time = np.random.uniform(0, 24)
            new_row = {'flight_id': self.flight_id, 'origin': origin, 'destination': destination, 'departure_time': departure_time, 'arrival_time': arrival_time}
            self.flights = pd.concat([self.flights, pd.DataFrame(new_row, index=[0])], ignore_index=True)
            self.flight_id += 1

airport = Airports(10, 100)

/tmp/ipykernel_131755/3809109187.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.airports = pd.concat([self.airports, pd.DataFrame(new_row, index=[0])], ignore_index=True)
/tmp/ipykernel_131755/3809109187.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.flights = pd.concat([self.flights, pd.DataFrame(new_row, index=[0])], ignore_index=True)


#  人员类 individual 和 机组人员类 staff

人员类需要具有的属性包括：
1. 姓名
2. 工号
3. 资质 1
4. 资质 2 
5. 资质 3
6. 资质 4
7. 组别(对应四个休息室)
8. 位置

人员类的方法
TODO 2024-03-21 待定

In [10]:
class individual():
    def __init__(self, unique_id, model):
        self.name = None
        # 增加基础属性
        self.isYiBan = False
        self.isLangQiao = False
        self.isQiaozai = False
        self.isZhongwen = False
        self.isYingyu = False
        self.workDuration = 0
        self.restPos = 183 # 183\60\70\39
        self.pos = 50 # 机位的选择
        self.isWork = False
    
    def step(self):
        print('individual step')

class staffSets(mesa.Agent):
    def __init__(self, num_individuals, airport):
        super().__init__()
        self.num_individuals = num_individuals
        self.individuals = pd.DataFrame(columns=['individual_id', 'name', 'isYiBan', 'isLangQiao', 'isQiaozai', 'isZhongwen', 'isYingyu', 'workDuration', 'restPos', 'pos', 'isWork'])
        
    
    def login(self,path):
        # 根据资质来注册员工列表
        pass

    def append_individual(self,individual):
        self.individuals.append(individual)
    
    def get_Idle_individual(self):
        # 获取空闲的员工
        pass

    
    def step(self):
        for individual in self.individuals:
            individual.step()

individual = individual(1, airport)